In [1]:
# imports
import os

import pandas as pd

In [2]:
# config
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission/"

In [3]:
# load data
df = pd.read_json(
    os.path.join(
        PROJECT_PATH, "Data/processed_data/dataset_wordplay_detection_propmts.json"
    ),
    orient="index"
)

In [6]:
df_dev = pd.read_json(
    os.path.join(
        PROJECT_PATH, "Data/processed_data/dev_dataset.json"
    ),
    orient="index"
)

In [7]:
df.head(2)

,user_prompt,system_prompt
0,Заголовок новости: Комфортная среда. Cодержани...,Присутствует ли в заголовке новости игра слов?...
1,Заголовок новости: Свинина стушевалась. Cодерж...,Присутствует ли в заголовке новости игра слов?...


In [8]:
df_dev.head(2)

,annotations,summary,is_word_play,date,article_url,headline,lead
0,[{'headline_substring': 'Театр божественных де...,В российский прокат выходит «Голда. Судный ден...,True,2023-08-31,https://www.kommersant.ru/doc/6173963,Театр божественных действий,«Голда»: история первых дней войны Судного дня
1,None,Участники переговоров по Украине в городе Джид...,False,2023-08-05,https://www.kommersant.ru/doc/6147213,Corriere della Sera: участники переговоров в Д...,На встрече в Джидде решили создать группы для ...


In [9]:
def extract_annotations_labels(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
            elif isinstance(annot, list):
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )
    
    return all_wordplay_type

In [10]:
df_dev["wordpla_type"] = df_dev["annotations"].apply(extract_annotations_labels)

In [12]:
set(df_dev["wordpla_type"].explode().tolist())

{'Collocation',
 'Homonymy',
 'Idiom',
 'Nonce word',
 'Oxymoron',
 'Phonetic similarity',
 'Polysemy',
 'Reference',
 nan}

In [13]:
collocations = list()
homonymys = list()
idioms = list()
nonce_words = list()
oxymorons = list()
phonetic_similaritys = list()
polysemys = list()
references = list()

In [23]:
collocations.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Collocation" in x)].sample(2)["headline"].tolist())
homonymys.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Homonymy" in x)].sample(2)["headline"].tolist())
idioms.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Idiom" in x)].sample(2)["headline"].tolist())
nonce_words.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Nonce word" in x)].sample(2)["headline"].tolist())
oxymorons.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Oxymoron" in x)].sample(2)["headline"].tolist())
phonetic_similaritys.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Phonetic similarity" in x)].sample(2)["headline"].tolist())
polysemys.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Polysemy" in x)].sample(2)["headline"].tolist())
references.extend(df_dev[df_dev["wordpla_type"].apply(lambda x: "Reference" in x)].sample(2)["headline"].tolist())

In [24]:
df["system_prompt"].iloc[0]

'Присутствует ли в заголовке новости игра слов? Дай ответ с учетом содержания новости. Отвечать можешь только "да", "нет" или "не знаю".'

In [26]:
sys_prompt_additional = f"""
Игрой слов считаются следующие механизмы:

1. Фонетический уровень

Механизм: Омофония
Определение: Слова, звучащие одинаково, но имеющие разное написание и значение.
Примеры:
    - {phonetic_similaritys[0]}
    - {phonetic_similaritys[1]}
Основные индикаторы: Идентичное произношение при различном написании.

2. Морфологический уровень

Механизм: Словообразование
Определение: Создание новых слов или выражений путём модификации или комбинации существующих.
Примеры:
    - {nonce_words[0]}
    - {nonce_words[1]}
Основные индикаторы: Необычные словесные конструкции, играющие с знакомыми
морфемами или частями слов.

3. Лексико-фразеологический уровень

Механизм: Трансформация фразеологизмов
Определение: Изменение известных фраз, идиом или пословиц для создания нового
смысла или комического эффекта.
Примеры:
    - {idioms[0]}
    - {idioms[1]}
Основные индикаторы: Изменения в фразеологизмах, сохраняющие некоторую структуру оригинальной фразы, но с изменёнными словами.

Механизм: Полисемия
Определение: Использование слов с несколькими значениями для создания двусмысленности или юмора.
Примеры:
    - {polysemys[0]}
    - {polysemys[1]}
Основные индикаторы: Одно слово в заголовке имеет более одного вероятного толкования.

Механизм: Омонимия
Определение: Слова, одинаковые по написанию и звучанию, но разные по значению.
Примеры:
    - {homonymys[0]}
    - {homonymys[1]}
Основные индикаторы: Идентичное написание и произношение с различным значением в контексте.

Механизм: Оксюморон
Определение: Сочетание противоположных по значению слов для создания парадоксального эффекта.
Примеры:
    - {oxymorons[0]}
    - {oxymorons[1]}
Основные индикаторы: Фразы, содержащие внутренне противоречивые идеи.

Механизм: Трансформация устойчивых сочетаний
Определение: Изменение или перестройка известных устойчивых сочетаний (терминов, выражений) для создания нового смысла или комического эффекта.
Примеры:
    - {collocations[0]}
    - {collocations[1]}
Основные индикаторы: Частичная замена или перестройка известных устойчивых сочетаний, которая сохраняет распознаваемую основу оригинала, но добавляет новый смысл или комический эффект.

4. Дискурсивный уровень
Механизм: Отсылка к прецедентным текстам
Определение: Ссылка на известные тексты, фильмы, песни или исторические события,
немного изменённые для нового контекста.
Примеры:
    - {references[0]}
    - {references[1]}
Основные индикаторы: Знание оригинального текста критично, заголовок часто вызывает коннотации оригинального произведения.

Дополнительно
Трансформация фразеологизма, пословицы, поговорки или прецедентного текста может осуществляться несколькими способами:
- с помощью замены одного из слов другим словом;
- с помощью вставки нового слова в середину устойчивого сочетания;
- с помощью добавления к одному из слов устойчивого сочетания приставки;
- с помощью отсечения приставки (обратный процесс);
- с помощью замены части звуков слова, то есть посредством подбора сходно звучащего слова. Слова могут иметь схожее окончание;
- с помощью усечения конца слов;
- с помощью изменения окончания.
"""

In [28]:
result_system_prompt = f"""{df["system_prompt"].iloc[0]} {sys_prompt_additional}"""

In [31]:
df["system_prompt"] = [result_system_prompt for _ in range(len(df))]

In [33]:
df[["user_prompt", "system_prompt"]].to_json(
    os.path.join(
        PROJECT_PATH, "Data/processed_data/dataset_wordplay_detection_propmts_extended.json"
    ),
    orient="index",
    force_ascii=False
)